In [1]:
# !pip install tensorflow

In [2]:
import tensorflow as tf
import os
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the input shape for the images (224x224 RGB images)
IMG_HEIGHT = 224
IMG_WIDTH = 224

input_shape = (IMG_HEIGHT, IMG_WIDTH, 3)

# Build the model
inputs = layers.Input(shape=input_shape)

# First Convolutional Block (7x7 Convolution with Stride 2, followed by Max Pooling)
x = layers.Conv2D(32, (7, 7), strides=(2, 2), activation='relu', padding='same')(inputs)
x = layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)

# Second Convolutional Block (3x3 Convolution with Stride 1, followed by Max Pooling)
x = layers.Conv2D(64, (3, 3), strides=(1, 1), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)

# Third Convolutional Block
x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)

# Fourth Convolutional Block
x = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)

# Global Average Pooling
x = layers.GlobalAveragePooling2D()(x)

# Dropout to prevent overfitting
x = layers.Dropout(0.3)(x)

# Fully Connected Layer with 10 units (corresponding to 10 classes)
NUM_CLASSES = 10
outputs = layers.Dense(NUM_CLASSES, activation='softmax')(x)

# Create the model
model = models.Model(inputs=inputs, outputs=outputs)

# Specify a lower learning rate (e.g., 0.0001)
LEARNING_RATE = 0.0001
optimizer = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)

# Compile the model with the specified learning rate
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Model Summary
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 112, 112, 32)   │         4,736 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 56, 56, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 56, 56, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 28, 28, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 28, 28, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 256)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 394,826 (1.51 MB)

 Trainable params: 394,826 (1.51 MB)

 Non-trainable params: 0 (0.00 B)

In [3]:
# Data Augmentation setup
train_datagen = ImageDataGenerator(
    rotation_range=20,          # Rotate images by up to 20 degrees
    width_shift_range=0.2,      # Shift images horizontally by up to 20%
    height_shift_range=0.2,     # Shift images vertically by up to 20%
    shear_range=0.2,            # Shear transformation
    zoom_range=0.2,             # Zoom in on images by up to 20%
    horizontal_flip=True,       # Randomly flip images horizontally
    fill_mode='nearest'         # Fill in missing pixels after transformations
)

# Load training data from directories
IMAGE_DIR = os.path.join(os.getcwd(), "..", "data", "imagenet_224")
BATCH_SIZE = 128
TARGET_SIZE = (IMG_WIDTH, IMG_HEIGHT)

# Image Dataset from Directory setup
# Automatically split the data into training (80%) and validation (20%)
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    IMAGE_DIR,           # Replace with the path to your image directory
    validation_split=0.2,       # Use 20% of the data for validation
    subset="training",          # Use the training split
    seed=42,                    # Set a seed to ensure repeatability
    image_size=TARGET_SIZE,      # Resize images to 224x224
    batch_size=BATCH_SIZE               # Batch size for training
)

validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    IMAGE_DIR,           # Same path to images
    validation_split=0.2,       # Use 20% of the data for validation
    subset="validation",        # Use the validation split
    seed=42,                    # Set a seed to ensure repeatability
    image_size=TARGET_SIZE,      # Resize images to 224x224
    batch_size=BATCH_SIZE               # Batch size for validation
)

Found 14738 files belonging to 10 classes.
Using 11791 files for training.
Found 14738 files belonging to 10 classes.
Using 2947 files for validation.


In [4]:
# Apply data augmentation to the training dataset
augmentation = tf.keras.Sequential([
    layers.RandomFlip('horizontal'),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.2),
])

# Apply the augmentation only on the training dataset
train_dataset = train_dataset.map(lambda x, y: (augmentation(x, training=True), y))

# Prefetch the datasets for performance improvement
train_dataset = train_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
# Train the model
EPOCHS = 30

history = model.fit(
    train_dataset,
    epochs=EPOCHS,                         # Adjust number of epochs as needed
    validation_data=validation_dataset
)

Epoch 1/30
93/93 ━━━━━━━━━━━━━━━━━━━━ 113s 1s/step - accuracy: 0.1387 - loss: 5.6075 - val_accuracy: 0.2874 - val_loss: 1.9784
Epoch 2/30
90/93 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/step - accuracy: 0.2684 - loss: 2.0086

In [ ]:
model.save(os.path.join("../data", "model_googlenet_simple"))

In [ ]:
import pickle

with open(os.path.join("../data", "history_googlenet_simple.pkl"), "wb") as file:
    pickle.dump(history, file)

In [ ]:
import matplotlib.pyplot as plt

def plot_accuracy_curve(training_result, metric):
    val_metric = f"val_{metric}"
    train_perf = training_result.history[metric]
    validation_perf = training_result.history[val_metric]
    
    plt.plot(train_perf, label=metric)
    plt.plot(validation_perf, label=val_metric)
    
    max_val = max(validation_perf)
    max_val_epoch = validation_perf.index(max_val)
    
    plt.xlabel("Epoch")
    plt.ylabel(metric)
    plt.legend(loc="lower right")
    
# plot_accuracy_curve(history, "accuracy")  
# plot_accuracy_curve(history, "categorical_accuracy")
plot_accuracy_curve(history, "top_k_categorical_accuracy")
# plot_accuracy_curve(history, "loss")